# import funcs and load hydrographics profiles ds

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate

import xarray as xr
import dask.array as da
from dask.distributed import Client

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches # for creating legend
import matplotlib.dates as mdates # converts datetime64 to datetime

import cartopy
import cartopy.crs as ccrs # for plotting
import cartopy.feature as cfeature # for map features
from cartopy.util import add_cyclic_point # for wrapping map fully - avoiding white line on 0 deg
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.dates as mdates # converts datetime64 to datetime
import matplotlib.gridspec as gridspec # to create grid-shaped combos of axes
from mpl_toolkits import mplot3d # 3d plotting tool
import cmocean # for nice oceanography colour pallettes

#import argopy
#from argopy import DataFetcher as ArgoDataFetcher # to load Argo ds directly

import os # for finding files

import gsw # for conversion functions

from tqdm.notebook import tqdm_notebook as tqdm
import glob # for downloading data
import sys # for path to functions

import seaborn as sns

sns.set(#font='Franklin Gothic Book',
        rc={
         'axes.axisbelow': False,
         'axes.edgecolor': 'Black',
         'axes.facecolor': 'w', 
                            # '#aeaeae',
         'axes.grid': False,
         'axes.labelcolor': 'k',
         'axes.spines.right': True,
         'axes.spines.top': True,
         'figure.facecolor': 'white',
         'lines.solid_capstyle': 'round',
         'patch.edgecolor': 'k',
         'patch.force_edgecolor': True,
         'text.color': 'k',
         'xtick.bottom': True,
         'xtick.color': 'k',
         'xtick.direction': 'out',
         'xtick.top': False,
         'ytick.color': 'k',
         'ytick.direction': 'out',
         'ytick.left': True,
         'ytick.right': False},
         font_scale=1)
mpl.rcParams["figure.titlesize"] = 30
mpl.rcParams["axes.titlesize"] = 25
mpl.rcParams["axes.labelsize"] = 20
mpl.rcParams["font.size"] = 12
mpl.rcParams["xtick.labelsize"] = 15
mpl.rcParams["ytick.labelsize"] = 15
mpl.rcParams["ytick.labelright"] = False

from warnings import filterwarnings as fw
fw('ignore')

# import my own funcs
import sys
sys.path.append('/home/theospira/notebooks/projects/WW_climatology/functions')
from plot_formatting import circular_boundary,plot_nice_box
from inspection_funcs import boxplot

import importlib
#importlib.reload(sys.modules['inspection_plot'])

In [2]:
#import importlib
#importlib.reload(sys.modules['load_data'])

path = '/home/theospira/notebooks/projects/WW_climatology'

import sys
sys.path.append(path+'/functions')
from plot_formatting import *
from inspection_funcs import boxplot
from smoothing_and_interp import *

## open profile ds and add z

In [4]:
ds1 = xr.open_dataset('/home/theospira/notebooks/projects/WW_climatology/data/hydrographic_profiles/superseded/ww_gauss_smoothed_ds-preDec23.nc')

# make ds that contains only WW profiles 
#ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]) 

for i,d in enumerate(np.unique(ds1.dsource)):
    idx = np.where(ds1.dsource == d)[0]
    ds1['dsource'][idx] = i+1

ds1.dsource.attrs['description'] = "'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"
ds1['dsource'] = ds1['dsource'].astype(int)

# calculate pressure variables as depth
arr = []
lat = ds1.lat.data
for v in ['ww_cp','up_bd','lw_bd']:
    arr += gsw.z_from_p(ds1[v],lat),

ds1['ww_cd'] = xr.DataArray(arr[0].copy()*-1)
ds1['up_bd'] = xr.DataArray(arr[1].copy()*-1)
ds1['lw_bd'] = xr.DataArray(arr[2].copy()*-1)

# calculate z across dataset
p = np.ndarray(ds1.ctemp.shape)*np.nan
l = np.ndarray(ds1.ctemp.shape)*np.nan

lat  = ds1.lat.data
pres = ds1.pres.data
for i in range(p.shape[0]):
    p[i,:] = pres
for i in range(l.shape[1]):
    l[:,i] = lat

ds1['z'] = xr.DataArray(gsw.z_from_p(p,l)*-1,dims=ds1.ctemp.dims)

In [3]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.stats as ss
from plot_formatting import *

def summer_ds_domain(ds,lon1=-40,lon2=-33,lat1=-58,lat2=-52):
    tmp = ds.groupby_bins(group='time.month',bins=range(0,15,3),labels=range(0,4))[0]
    idx = np.where((np.logical_and(tmp.lon>lon1,tmp.lon<lon2) & np.logical_and(tmp.lat>lat1,tmp.lat<lat2)))
    return tmp.isel(n_prof=idx[0])

def plot_profile(a,tmp3,tmp4,v):
    z = gsw.z_from_p(tmp3.pres,tmp.lat.mean())*-1
    a.plot(tmp3[v],tmp3.z,c='#0c0cff',label='Mean')
    a.plot((tmp3[v]-tmp4[v]),tmp3.z,c='#aec6cf',label='Std')
    a.plot((tmp3[v]+tmp4[v]),tmp3.z,c='#aec6cf',)
    a.axhline(tmp3.up_bd,ls='--',lw=2,c='#666666',label='Upper/lower\nboundary')
    a.axhline(tmp3.ww_cd,ls='--',lw=2,c='black',label='WW core')
    a.axhline(tmp3.lw_bd,ls='--',lw=2,c='#666666')

In [7]:
tmp  = summer_ds_domain(ds1)
tmp['n2'] = tmp.n2*1e5
tmp3 = tmp.mean('n_prof')  # for mean profiles
tmp4 = tmp.std('n_prof')   # for std  profiles

In [9]:
def domain_profiles(tmp,lon1=-40,lon2=-33,lat1=-58,lat2=-52):
    idx = np.where((np.logical_and(tmp.lon>lon1,tmp.lon<lon2) & np.logical_and(tmp.lat>lat1,tmp.lat<lat2)))
    return idx[0]

In [14]:
# select all profiles in box domain:
idx = domain_profiles(ds1,)
ds_bx = ds1.isel(n_prof=idx)

In [34]:
idx = domain_profiles(ds1,lon1=-37,lon2=-31,lat1=-58,lat2=-55)
ds_bx = ds1.isel(n_prof=idx)

In [41]:
ds_bx.ww_n2

<xarray.DataArray 'ww_n2' (n_prof: 798)>
[798 values with dtype=float64]
Coordinates:
  * n_prof   (n_prof) int64 5300 5623 21328 21655 ... 608954 619385 619902
    lon      (n_prof) float64 -35.05 -34.06 -36.1 -34.51 ... -33.44 -36.78 -35.5
    lat      (n_prof) float64 -56.44 -55.83 -56.16 ... -55.82 -55.78 -56.24
    time     (n_prof) datetime64[ns] 2005-02-18T06:31:50.000000256 ... 2021-1...

In [42]:
def month_idx_sel(ds1,i):
    idx = np.where(ds1.time.dt.month==i)[0]
    return idx

In [45]:
def year_idx_sel(ds1,yr):
    idx = np.where(chk.time.dt.year==yr)[0]
    return idx

In [52]:
ds_bx.isel(n_prof=np.where(ds_bx.time.dt.year==2021)[0])

<xarray.Dataset>
Dimensions:  (n_prof: 35, pres: 196)
Coordinates:
  * n_prof   (n_prof) int64 601794 602181 602233 602238 ... 608954 619385 619902
  * pres     (pres) int64 10 12 14 16 18 20 22 ... 388 390 392 394 396 398 400
    lon      (n_prof) float64 -36.41 -36.67 -34.85 ... -33.44 -36.78 -35.5
    lat      (n_prof) float64 -57.85 -55.5 -56.7 -56.7 ... -55.82 -55.78 -56.24
    time     (n_prof) datetime64[ns] 2021-02-07T04:50:56 ... 2021-12-23T05:45...
Data variables: (12/22)
    temp     (n_prof, pres) float64 ...
    psal     (n_prof, pres) float64 ...
    dsource  (n_prof) int64 1 2 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 1 1 1 1
    mld      (n_prof) float64 ...
    asal     (n_prof, pres) float32 ...
    ctemp    (n_prof, pres) float32 ...
    ...       ...
    ww_type  (n_prof) float64 ...
    thcc     (n_prof) float64 ...
    ww_n2    (n_prof) float64 ...
    ww_msk   (n_prof, pres) float64 ...
    ww_cd    (n_prof) float64 95.09 95.11 101.0 97.08 ... 134.7 150.6 174.3
    z        (n_prof, pres) float64 9.907 11.89 13.87 ... 392.0 394.0 396.0
Attributes:
    description:  Interpolated dataset from 2004 to 2021. Contains MLD from p...

In [44]:
for i in range(3):
    print(i+1,month_idx_sel(ds_bx,i+1).size)

1 96
2 148
3 51


In [50]:
for i in range(3):
    print(i+1,year_idx_sel(ds1.isel(n_prof=month_idx_sel(ds_bx,i+1)),2021).size)

1 30
2 30
3 30


### save data for Tasha 

In [21]:
chk = xr.open_dataset('data/box-domain-JFM-profiles.nc')

In [29]:
jan = xr.open_dataset('data/SO-hydrographic-profiles-jan-2021.nc')
jan.lat.data

array([ 129.6709 ,   84.622  ,  159.09132, ..., -165.93615,  -88.76772,
       -107.13916])

In [24]:
for i in range(3):
    print(i+1,month_idx_sel(chk,i+1).size)

1 397
2 425
3 294


In [13]:
tmp.to_netcdf('box-domain-JFM-profiles.nc')

In [31]:
for i,m in enumerate(['jan','feb','mar','apr']):
    idx = np.where(np.logical_and(ds1.time.dt.month==i+1,ds1.time.dt.year==2021))[0]
    ds_tmp = ds1.isel(n_prof=idx)
    ds_tmp.to_netcdf(f'data/SO-hydrographic-profiles-{m}-2021.nc')

In [42]:
vars = ['ctemp',
         'asal',
         'sig',
         'n2',
         'mld',
         'up_bd',
         'lw_bd',
         'ww_cd',
         'ww_ct',
         'ww_sa',
         'thcc',
         'ww_n2',
         'sig_c',
         'ww_type',
         'dsource',
         'n_prof',
         'ww_prof',
         'ww_msk',]

In [57]:
ds_sum = ds.isel(season=2)[vars]
ds_sum['mld'] = ds_sum['mld'] * -1
del(ds_sum['season'])

In [58]:
ds_sum.to_netcdf('data/SO_JFM_climatology-1_deg.nc')

In [59]:
ds_sum

<xarray.Dataset>
Dimensions:  (lon: 360, lat: 40, pres: 146)
Coordinates:
  * lon      (lon) int64 -180 -179 -178 -177 -176 -175 ... 175 176 177 178 179
  * lat      (lat) int64 -79 -78 -77 -76 -75 -74 -73 ... -45 -44 -43 -42 -41 -40
  * pres     (pres) int64 10 12 14 16 18 20 22 ... 288 290 292 294 296 298 300
Data variables: (12/18)
    ctemp    (lon, lat, pres) float64 nan nan nan nan ... 12.04 11.99 11.94
    asal     (lon, lat, pres) float64 nan nan nan nan ... 35.18 35.18 35.18
    sig      (lon, lat, pres) float64 nan nan nan nan ... 26.58 26.58 26.58
    n2       (lon, lat, pres) float64 nan nan nan ... 1.167e-05 1.167e-05
    mld      (lon, lat) float64 nan nan 9.895 nan 93.0 ... nan nan nan nan nan
    up_bd    (lon, lat) float64 nan nan 9.895 nan 50.46 ... nan nan nan nan nan
    ...       ...
    sig_c    (lon, lat) float64 nan nan 27.13 nan 27.74 ... nan nan nan nan nan
    ww_type  (lon, lat) float64 nan nan 2.0 nan 1.5 nan ... nan nan nan nan nan
    dsource  (lon, lat) float64 nan nan 4.0 nan 4.0 nan ... 1.0 nan 1.0 1.0 1.0
    n_prof   (lon, lat) float64 nan nan 10.0 nan 7.0 ... 9.0 nan 14.0 21.0 19.0
    ww_prof  (lon, lat) float64 nan nan 10.0 nan 4.0 nan ... nan nan nan nan nan
    ww_msk   (lon, lat, pres) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    description:  1 degree grid of median climatology (season) along 2dbar pr...

# run gridding for 0.5°x0.5°

## make sure all gs = 0.5

In [3]:
def grid_lat_3d(dsgpd_ln,gs=0.5):
    lat_min = -80
    lat_max = -40
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = np.arange(0,lat_max-lat_min,gs)
    
    return dsgpd_ln.groupby_bins('lat',lat,
                       labels=lat_labels,
                       restore_coord_dims=True).median(skipna=True,dim='n_prof')
    
def grid_lon_3d(dsgpd_t,gs=0.5):
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = np.arange(0,lon_max-lon_min,gs)

    return dsgpd_t.groupby_bins('lon',lon,
                       labels=lon_labels,
                       restore_coord_dims=True).apply(grid_lat_3d)
    
def grid_var_3d(dsvar,clim='month',gs=0.5):
    """for gridding spatially in 2D and time (3D)."""
    if clim == 'season':
        var = dsvar.groupby_bins(group='time.month',bins=range(0,15,3),labels=range(0,4)).apply(grid_lon_3d)
    else:
        var = dsvar.groupby('time.'+clim).apply(grid_lon_3d)
    return var

In [4]:
def grid_lat_3d_sum(dsgpd_ln,gs=0.5):
    lat_min = -80
    lat_max = -40
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = np.arange(0,lat_max-lat_min,gs)
    
    return dsgpd_ln.groupby_bins('lat',lat,
                       labels=lat_labels,
                       restore_coord_dims=True).sum(skipna=True,dim='n_prof')
    
def grid_lon_3d_sum(dsgpd_t,gs=0.5):
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = np.arange(0,lon_max-lon_min,gs)

    return dsgpd_t.groupby_bins('lon',lon,
                       labels=lon_labels,
                       restore_coord_dims=True).apply(grid_lat_3d_sum)
    
def grid_var_3d_sum(ds,dsvar,clim='month',gs=0.5):
    """for gridding spatially in 2D and time (3D). take the sum of the grid cell (for number of profiles)"""
    ds[dsvar] = np.ones(ds[dsvar].shape)
    if clim == 'season':
        var = ds[dsvar].groupby_bins(group='time.month',bins=range(0,15,3),labels=range(0,4)).apply(grid_lon_3d_sum)
    else:
        var = ds[dsvar].groupby('time.'+clim).apply(grid_lon_3d_sum)
    return var

In [5]:
def grid_lat_4d(dsgpd_ln,gs=0.5):
    lat_min = -80
    lat_max = -40
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = np.arange(0,lat_max-lat_min,gs)
    
    return dsgpd_ln.groupby_bins('lat',lat,
                       labels=lat_labels,
                       restore_coord_dims=True).median(skipna=True,dim='n_prof')
    
def grid_lon_4d(dsgpd_t,gs=0.5):
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = np.arange(0,lon_max-lon_min,gs)

    return dsgpd_t.groupby_bins('lon',lon,
                       labels=lon_labels,
                       restore_coord_dims=True).apply(grid_lat_4d)
    
def grid_var_4d(dsvar,clim='month',gs=0.5):
    """for gridding spatially in 3D and in time (4D)."""
    if clim == 'season':
        var = dsvar.groupby_bins(group='time.month',bins=range(0,15,3),labels=range(0,4)).apply(grid_lon_4d)
    else:
        var = dsvar.groupby('time.'+clim).apply(grid_lon_4d)
    return var

In [6]:
from scipy.stats import mode

def mode_gridding_month(dsvar,gs=0.5):
        
    # define lon min and max resp
    lon_min = -180
    lon_max = 180
    lon = np.arange(lon_min,lon_max+gs,gs)
    lon_labels = np.arange(0,lon_max-lon_min,gs)
    
    lat_min = -80
    lat_max = -40
    lat = np.arange(lat_min,lat_max+gs,gs)
    lat_labels = np.arange(0,lat_max-lat_min,gs)
    
    # array that we will map our data to (gs°xgs° gridding)
    arr = np.ndarray([12,lon_labels.size,lat_labels.size])*np.nan 
    
    # group by seasons
    var = dsvar.groupby(dsvar.time.dt.month)
    
    # group into lon bins
    for t in tqdm(list(var.groups.keys())):
        var1 = var[t].groupby_bins('lon',lon,labels=lon_labels,restore_coord_dims=True)
        
        # now group into lat bins for each lon group:
        for lt_idx,i in enumerate(list(var1.groups.keys())):
            var2 = var1[i].groupby_bins('lat',lat,labels=lat_labels,restore_coord_dims=True)
            
            # now take the mode!
            for ln_idx,l in enumerate(list(var2.groups.keys())):
                arr[t-1,lt_idx-1,ln_idx-1] = mode(var2[l],nan_policy='omit')[0]
                
    return arr

## run seasonal gridding

In [8]:
ds = xr.open_dataset('/home/theospira/notebooks/projects/WW_climatology/data/hydrographic_profiles/superseded/ww_gauss_smoothed_ds-preDec23.nc')
#ds = xr.open_dataset('/home/theospira/notebooks/projects/WW_climatology/data/hydrographic_profiles/ww_gauss_smoothed_ds.nc')

for i,d in enumerate(np.unique(ds.dsource)):
    idx = np.where(ds.dsource == d)[0]
    ds['dsource'][idx] = i+1

ds.dsource.attrs['description'] = "'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"
ds['dsource'] = ds['dsource'].astype(int)

ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]) # make ds that contains only WW profiles 

In [11]:
clim = 'season'

dvars_4 = [] # 4D vars
for i in tqdm(['asal', 'ctemp', 'rho','sig','n2']):
    dvars_4 += grid_var_4d(ds[i],clim, gs = 0.5),

ww_vars = ['mlp','up_bd','ww_cp','ww_ct','ww_sa','lw_bd','thcc','ww_n2','sig_c']

dvars_3 = [] # 3D vars
for i in tqdm(ww_vars):
    dvars_3 += grid_var_3d(ds_ww[i],clim,gs=0.5),
n_prof  = grid_var_3d_sum(ds,'n_prof',clim,gs=0.5)
ww_prof = grid_var_3d_sum(ds_ww,'n_prof',clim,gs=0.5)
dsource = mode_gridding_szn(ds['dsource'],gs=0.5)
ww_type = mode_gridding_szn(ds['ww_type'],gs=0.5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

NameError: name 'mode_gridding_szn' is not defined

In [14]:
dvars_3 = [] # 3D vars
for i in tqdm(ww_vars):
    dvars_3 += grid_var_3d(ds_ww[i],clim,gs=0.5),
n_prof  = grid_var_3d_sum(ds,'n_prof',clim,gs=0.5)
ww_prof = grid_var_3d_sum(ds_ww,'n_prof',clim,gs=0.5)
dsource = mode_gridding_szn(ds['dsource'],gs=0.5)
ww_type = mode_gridding_szn(ds['ww_type'],gs=0.5)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
# create dataset
x    = 0.5
time = np.arange(0,4,1) # JFM,AMJ,JAS,OND: summer,...,spring
p    = ds.pres.data
lat  = np.arange(-79.5,-40+x,x)
lon  = np.arange(-180,180,x)

ds_grid = xr.Dataset(
            data_vars = dict(
                            ctemp   = (['time','pres','lon','lat'], dvars_4[1].data),
                            asal    = (['time','pres','lon','lat'], dvars_4[0].data),
                            rho     = (['time','pres','lon','lat'], dvars_4[2].data),
                            sig     = (['time','pres','lon','lat'], dvars_4[3].data),
                            n2      = (['time','pres','lon','lat'], dvars_4[4].data),
                            mlp     = (['time', 'lon','lat'], dvars_3[0].data),
                            up_bd   = (['time', 'lon','lat'], dvars_3[1].data),
                            lw_bd   = (['time', 'lon','lat'], dvars_3[5].data),
                            ww_cp   = (['time', 'lon','lat'], dvars_3[2].data),
                            ww_ct   = (['time', 'lon','lat'], dvars_3[3].data),
                            ww_sa   = (['time', 'lon','lat'], dvars_3[4].data),
                            thcc    = (['time', 'lon','lat'], dvars_3[6].data),
                            ww_n2   = (['time', 'lon','lat'], dvars_3[7].data),
                            sig_c   = (['time', 'lon','lat'], dvars_3[8].data),
                            ww_type = (['time', 'lon','lat'], ww_type.data),
                            dsource = (['time', 'lon','lat'], dsource.data),
                            n_prof  = (['time', 'lon','lat'], n_prof.data),
                            ww_prof = (['time', 'lon','lat'], ww_prof.data),                
                            ),
            coords   = dict(
                            time    = (['time'], time),
                            lon     = (['lon'], lon),
                            lat     = (['lat'], lat),
                            pres    = (['pres'], p)
                            ),
            attrs    = dict(description=str(
    f"{x}° grid of median climatology ({clim}) along 2dbar pressure interpolated and quality controlled Argo, MEOP, SOCCOM, CTDs, and Gliders data from 2004 to 2021. Contains good QC and Wilson et al (2019) QC. Smoothed temp and psal using Gauss smoothing with stand dev=2. Gsw funcs and WW subsequently calculated. Further QC post calculation: NaN any WW vars profiles with a lower boundary deeper than 300dbar; drop any profiles with fewer than 10 datapoints. Removed vertical profiles by cross referencing temperature profile with mixed layer temperature. If there is not a significant change from the mean ML temperature, we remove the profile (a change of 0.25°C or more for at least 5% of the profile)."))
            )

# sort some stuff
ds_grid.n_prof.attrs['description']  = 'total number of profiles across the time span for that month.'
ds_grid.ww_prof.attrs['description'] = 'total number of profiles across the time span for that month containing Winter Water.'
ds_grid.ww_type.attrs['description'] = 'mode of ww classification per grid cell. key: 1:ML WW, 2:Subsurface WW'
ds_grid.dsource.attrs['description'] = "mode of data source. key: 'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"

# reorder seasons
ds_grid['time'] = [2,3,0,1]
ds_grid = ds_grid.sortby('time')
ds_grid.time.attrs['description'] = '0:Winter, 1:Spring, 2:Summer, 3:Autumn'

In [16]:
# save that mf dataset
ds_grid.to_netcdf('SO_1yr_clim_seasonal-0.5_deg_v2.nc')

## run monthly gridding without the years of 2004,2015 & 2021

In [5]:
ds1 = xr.open_dataset('/home/theospira/notebooks/projects/WW_climatology/data/hydrographic_profiles/superseded/ww_gauss_smoothed_ds-preDec23.nc')

# make ds that contains only WW profiles 
#ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]) 

for i,d in enumerate(np.unique(ds1.dsource)):
    idx = np.where(ds1.dsource == d)[0]
    ds1['dsource'][idx] = i+1

ds1.dsource.attrs['description'] = "'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"
ds1['dsource'] = ds1['dsource'].astype(int)

# calculate pressure variables as depth
arr = []
lat = ds1.lat.data
for v in ['ww_cp','up_bd','lw_bd']:
    arr += gsw.z_from_p(ds1[v],lat),

ds1['ww_cd'] = xr.DataArray(arr[0].copy()*-1)
ds1['up_bd'] = xr.DataArray(arr[1].copy()*-1)
ds1['lw_bd'] = xr.DataArray(arr[2].copy()*-1)

# calculate z across dataset
p = np.ndarray(ds1.ctemp.shape)*np.nan
l = np.ndarray(ds1.ctemp.shape)*np.nan

lat  = ds1.lat.data
pres = ds1.pres.data
for i in range(p.shape[0]):
    p[i,:] = pres
for i in range(l.shape[1]):
    l[:,i] = lat

ds1['z'] = xr.DataArray(gsw.z_from_p(p,l)*-1,dims=ds1.ctemp.dims)

In [6]:
def flatten_list(matrix):
    return [item for row in matrix for item in row]

def ds_index_year(yr,ds1):
    idx = []
    if type(yr)==int:
        yr = [yr]
    for i in yr: #2004,2015,2021]:
        idx += np.where(ds1.time.dt.year==i)[0],
    idx = flatten_list(idx)
    
    ds = ds1.isel(n_prof=idx)
    
    ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]).copy() # make ds that contains only WW profiles 

    return ds, ds_ww

In [9]:
clim = 'month'


for y in tqdm([2004,2015,2021],"dataset"):
#for y in tqdm([2021],"dataset"):
    ds,ds_ww = ds_index_year(y,ds1)

    # grid dataset
    dvars_4 = [] # 4D vars
    for i in tqdm(['asal', 'ctemp', 'rho','sig','n2','z']):
        dvars_4 += grid_var_4d(ds[i],clim, gs = 0.5),
    
    ww_vars = ['mld','up_bd','ww_cd','ww_ct','ww_sa','lw_bd','thcc','ww_n2','sig_c']
    
    dvars_3 = [] # 3D vars
    for i in tqdm(ww_vars):
        dvars_3 += grid_var_3d(ds_ww[i],clim,gs=0.5),
    n_prof  = grid_var_3d_sum(ds,'n_prof',clim,gs=0.5)
    ww_prof = grid_var_3d_sum(ds_ww,'n_prof',clim,gs=0.5)
    dsource = mode_gridding_month(ds['dsource'],gs=0.5)
    ww_type = mode_gridding_month(ds['ww_type'],gs=0.5)

    # create dataset
    x    = 0.5
    time = np.arange(1,13,1) # months: JFMAMJJASOND
    p    = ds.pres.data
    lat  = np.arange(-79.5,-40+x,x)
    lon  = np.arange(-180,180,x)
    
    ds_grid = xr.Dataset(
                data_vars = dict(
                                ctemp   = (['time','pres','lon','lat'], dvars_4[1].data),
                                asal    = (['time','pres','lon','lat'], dvars_4[0].data),
                                rho     = (['time','pres','lon','lat'], dvars_4[2].data),
                                sig     = (['time','pres','lon','lat'], dvars_4[3].data),
                                n2      = (['time','pres','lon','lat'], dvars_4[4].data),
                                z       = (['time','pres','lon','lat'], dvars_4[5].data),
                                mld     = (['time', 'lon','lat'], dvars_3[0].data),
                                up_bd   = (['time', 'lon','lat'], dvars_3[1].data),
                                lw_bd   = (['time', 'lon','lat'], dvars_3[5].data),
                                ww_cd   = (['time', 'lon','lat'], dvars_3[2].data),
                                ww_ct   = (['time', 'lon','lat'], dvars_3[3].data),
                                ww_sa   = (['time', 'lon','lat'], dvars_3[4].data),
                                thcc    = (['time', 'lon','lat'], dvars_3[6].data),
                                ww_n2   = (['time', 'lon','lat'], dvars_3[7].data),
                                sig_c   = (['time', 'lon','lat'], dvars_3[8].data),
                                ww_type = (['time', 'lon','lat'], ww_type.data),
                                dsource = (['time', 'lon','lat'], dsource.data),
                                n_prof  = (['time', 'lon','lat'], n_prof.data),
                                ww_prof = (['time', 'lon','lat'], ww_prof.data),                
                                ),
                coords   = dict(
                                time    = (['time'], time),
                                lon     = (['lon'], lon),
                                lat     = (['lat'], lat),
                                pres    = (['pres'], p)
                                ),
                attrs    = dict(description=str(
        f"{x}° grid of median climatology ({clim}) along 2dbar pressure interpolated and quality controlled Argo, MEOP, SOCCOM, CTDs, and Gliders data from 2004 to 2021. Contains good QC and Wilson et al (2019) QC. Smoothed temp and psal using Gauss smoothing with stand dev=2. Gsw funcs and WW subsequently calculated. Further QC post calculation: NaN any WW vars profiles with a lower boundary deeper than 300dbar; drop any profiles with fewer than 10 datapoints. Removed vertical profiles by cross referencing temperature profile with mixed layer temperature. If there is not a significant change from the mean ML temperature, we remove the profile (a change of 0.25°C or more for at least 5% of the profile)."))
                )
    
    # sort some stuff
    ds_grid.n_prof.attrs['description']  = 'total number of profiles across the time span for that month.'
    ds_grid.ww_prof.attrs['description'] = 'total number of profiles across the time span for that month containing Winter Water.'
    ds_grid.ww_type.attrs['description'] = 'mode of ww classification per grid cell. key: 1:ML WW, 2:Subsurface WW'
    ds_grid.dsource.attrs['description'] = "mode of data source. key: 'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"
    
    # reorder seasons
    #ds_grid['time'] = [2,3,0,1]
    #ds_grid = ds_grid.sortby('time')
    ds_grid.time.attrs['description'] = 'Months: JFMAMJJASOND'
    
    # save that mf dataset
    ds_grid.to_netcdf(f'data/SO_1yr_clim_monthly-0.5_deg-{y}.nc')
    
    m = ['J','F','M']
    for i in range(0,3,1):
        ds_grid.isel(time=i).to_netcdf(f'data/SO_clim_{m[i]}-0.5_deg-{y}.nc')

dataset:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
# grid all years but remove the years of 2004, 2015, 2021

clim = 'month'

idx = []
for i in [2004,2015,2021]:
    idx += np.where(ds1.time.dt.year==i)[0],
idx = flatten_list(idx)

ds = ds1.drop_isel(n_prof=idx)
# make ds that contains only WW profiles 
ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]).copy() 


# grid dataset
dvars_4 = [] # 4D vars
for i in tqdm(['asal', 'ctemp', 'rho','sig','n2','z']):
    dvars_4 += grid_var_4d(ds[i],clim, gs = 0.5),

ww_vars = ['mld','up_bd','ww_cd','ww_ct','ww_sa','lw_bd','thcc','ww_n2','sig_c']

dvars_3 = [] # 3D vars
for i in tqdm(ww_vars):
    dvars_3 += grid_var_3d(ds_ww[i],clim,gs=0.5),
n_prof  = grid_var_3d_sum(ds,'n_prof',clim,gs=0.5)
ww_prof = grid_var_3d_sum(ds_ww,'n_prof',clim,gs=0.5)
dsource = mode_gridding_month(ds['dsource'],gs=0.5)
ww_type = mode_gridding_month(ds['ww_type'],gs=0.5)

# create dataset
x    = 0.5
time = np.arange(1,13,1) # months: JFMAMJJASOND
p    = ds.pres.data
lat  = np.arange(-79.5,-40+x,x)
lon  = np.arange(-180,180,x)

ds_grid = xr.Dataset(
            data_vars = dict(
                            ctemp   = (['time','pres','lon','lat'], dvars_4[1].data),
                            asal    = (['time','pres','lon','lat'], dvars_4[0].data),
                            rho     = (['time','pres','lon','lat'], dvars_4[2].data),
                            sig     = (['time','pres','lon','lat'], dvars_4[3].data),
                            n2      = (['time','pres','lon','lat'], dvars_4[4].data),
                            z       = (['time','pres','lon','lat'], dvars_4[5].data),
                            mld     = (['time', 'lon','lat'], dvars_3[0].data),
                            up_bd   = (['time', 'lon','lat'], dvars_3[1].data),
                            lw_bd   = (['time', 'lon','lat'], dvars_3[5].data),
                            ww_cd   = (['time', 'lon','lat'], dvars_3[2].data),
                            ww_ct   = (['time', 'lon','lat'], dvars_3[3].data),
                            ww_sa   = (['time', 'lon','lat'], dvars_3[4].data),
                            thcc    = (['time', 'lon','lat'], dvars_3[6].data),
                            ww_n2   = (['time', 'lon','lat'], dvars_3[7].data),
                            sig_c   = (['time', 'lon','lat'], dvars_3[8].data),
                            ww_type = (['time', 'lon','lat'], ww_type.data),
                            dsource = (['time', 'lon','lat'], dsource.data),
                            n_prof  = (['time', 'lon','lat'], n_prof.data),
                            ww_prof = (['time', 'lon','lat'], ww_prof.data),                
                            ),
            coords   = dict(
                            time    = (['time'], time),
                            lon     = (['lon'], lon),
                            lat     = (['lat'], lat),
                            pres    = (['pres'], p)
                            ),
            attrs    = dict(description=str(
    f"{x}° grid of median climatology ({clim}) along 2dbar pressure interpolated and quality controlled Argo, MEOP, SOCCOM, CTDs, and Gliders data from 2004 to 2021. Contains good QC and Wilson et al (2019) QC. Smoothed temp and psal using Gauss smoothing with stand dev=2. Gsw funcs and WW subsequently calculated. Further QC post calculation: NaN any WW vars profiles with a lower boundary deeper than 300dbar; drop any profiles with fewer than 10 datapoints. Removed vertical profiles by cross referencing temperature profile with mixed layer temperature. If there is not a significant change from the mean ML temperature, we remove the profile (a change of 0.25°C or more for at least 5% of the profile)."))
            )

# sort some stuff
ds_grid.n_prof.attrs['description']  = 'total number of profiles across the time span for that month.'
ds_grid.ww_prof.attrs['description'] = 'total number of profiles across the time span for that month containing Winter Water.'
ds_grid.ww_type.attrs['description'] = 'mode of ww classification per grid cell. key: 1:ML WW, 2:Subsurface WW'
ds_grid.dsource.attrs['description'] = "mode of data source. key: 'Argo':1, 'CTD':2, 'Gliders':3, 'MEOP':4, 'SOCCOM':5"

# reorder seasons
#ds_grid['time'] = [2,3,0,1]
#ds_grid = ds_grid.sortby('time')
ds_grid.time.attrs['description'] = 'Months: JFMAMJJASOND'

# save that mf dataset
ds_grid.to_netcdf(f'data/SO_1yr_clim_monthly-0.5_deg-no_2004_2015_2021.nc')

m = ['J','F','M']
for i in range(0,3,1):
    ds_grid.isel(time=i).to_netcdf(f'data/SO_clim_{m[i]}-0.5_deg-no_2004_2015_2021.nc')

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

# regrid for median of ww n2 for the grouped months of JFMA

In [ ]:
# grid all years but remove the years of 2004, 2015, 2021

clim = 'month'

idx = []
for i in [2004,2015,2021]:
    idx += np.where(ds1.time.dt.year==i)[0],
idx = flatten_list(idx)

ds = ds1.drop_isel(n_prof=idx)
# make ds that contains only WW profiles 
ds_ww = ds.isel(n_prof=np.where(ds.ww_type.notnull())[0]).copy() 

# select months of JFMA
idx=[]
for i in [1,2,3,4]:
    idx += np.where(ds_ww.time.dt.month==i)[0],
idx = flatten_list(idx)
smr = ds_ww.isel(n_prof=idx)
smr

# grid ww n2
var = grid_lon_3d(smr.ww_n2)

x    = 0.5
lat  = np.arange(-79.5,-40+x,x)
lon  = np.arange(-180,180,x)

var = var.rename({'lon_bins':'lon','lat_bins':'lat'})
var['lat'] = lat
var['lon'] = lon

var.to_netcdf('data/SO_JFMA_clim-ww_n2-0.5_deg-no_2004_2015_2021.nc')

In [44]:
ds_grid = xr.open_dataset('data/SO_JFMA_clim-ww_n2-0.5_deg-no_2004_2015_2021.nc')

In [45]:
arr = []

In [50]:
grid_lon_3d(smr[v])

<xarray.DataArray 'mld' (lon_bins: 720, lat_bins: 80)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat_bins  (lat_bins) float64 0.0 0.5 1.0 1.5 2.0 ... 38.0 38.5 39.0 39.5
  * lon_bins  (lon_bins) float64 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [51]:
# add the rest of the ww variables to the dataset
ww_vars = ['mld','up_bd','ww_cd','ww_ct','ww_sa','lw_bd','thcc','sig_c']

for v in tqdm(ww_vars):
    ds_grid[v] = xr.DataArray(grid_lon_3d(smr[v]).data,dims=["lon","lat"])

  0%|          | 0/8 [00:00<?, ?it/s]

### note: deleted file data/SO_JFMA_clim-ww_n2-0.5_deg-no_2004_2015_2021.nc here 

In [53]:
ds_grid.to_netcdf('data/SO_JFMA_clim-ww_n2-0.5_deg-no_2004_2015_2021.nc')